# Necessary imports

In [ ]:
%pip install pandas
%pip install numpy
%pip install scikit-learn

import pandas as pd
import numpy as np
from datetime import datetime, time

# Carregamento de dados

In [ ]:
dtypes = {
    "hora_minuto":  str,
    "municipio": "category",
    "bairro": "category",
    "endereco": "category",
    "origem_chamado": "category",
    "tipo": "category",
    "subtipo": "category",
    "sexo": "category",
    "idade": float,
    "motivo_finalizacao": "category",
    "motivo_desfecho": "category",
}

columns_to_datetime = ["data"]

raw_df = pd.read_csv("./datasets/ocorrencias2022.csv", sep=';', dtype=dtypes, parse_dates=columns_to_datetime)
raw_df['hora_minuto'] = pd.to_datetime(raw_df['hora_minuto']).dt.time

# Tratando dados vazios e inconsistentes

In [ ]:
raw_df = raw_df.drop("motivo_finalizacao", axis=1)
raw_df = raw_df.dropna(subset=["municipio", "bairro", "subtipo", "sexo", "idade"])

def older_than_120(age: int):
  if age >= 120:
    return 120
  return age

raw_df["idade"] = raw_df["idade"].apply(lambda x : older_than_120(x))

In [ ]:
raw_df.head()

In [ ]:
motivos_desfecho = raw_df['motivo_desfecho'].value_counts()
motivos_desfecho

# Propósito do modelo

Quando um chamado é iniciado, a maior partes das informações é recolhida na hora. O motivo do desfecho, contudo, só pode ser preenchido após o encerramento do chamado. Destacamos os seguintes motivos de desfecho:

"PACIENTE JÉ ENCONTRADO EM ÓBITO"

"ÓBITO DURANTE O ATENDIMENTO"

Partimos do pressuposto de que alguns desses casos de óbito poderiam ser evitados com maior agilidade ou priorização por parte do SAMU.

Daí veio a idéia do nosso modelo:

Um modelo capaz de determinar com certo grau de certeza, baseando-se nos detalhes recolhidos na hora do registro da ocorrência, se aquela ocorrência corre risco de terminar com algum óbito. Caso ela afirme positivamente, essa informação poderia ser usada para maior priorização ou agilidade por parte da equipe.

# Criação do dataset de treino

1. Inicialmente, iremos criar uma coluna de "obito" mais simples, que engloba os 2 tipos de motivo de desfecho que levaram a óbitos, é composta por 1 ou 0, em caso de óito ou o contrário.

2. Após isso, removeremos a coluna "motivo_desfecho", pois, como discutido, ela só é preenchida após a conclusão da ocorrência, então não faria sentido o modelo ter acesso a essa informação no momento em que analisa uma ocorrência nova.

3. Criaremos uma coluna que transforme o dado de hora numa relação mais categórica e genérica, "Período", como madrugada, manhã, tarde e noite.

4. Removeremos colunas que julgamos serem irrelevantes para a classificação, como data e colunas de endereço. Decidimos manter a coluna de hora_minuto pois é argumentável que a hora que algo ocorre pode impactar na conclusão.

4. Para utilização do scikit learn, utilizaremos a técnica One Hot Enconding para transformar cada tipo categórico em um formato mais desejável para o scikit learn.
É importante ressaltar que os dados categóricos "Período" e "Idade" são ordinais, isso é, uma certa ordem pode ser determinada entre as categorias (manhã vem antes da tarde que vem antes da noite, 55 anos vem antes dos 56 anos), porém o resto das colunas representam dados nominais.

5. Por fim, separamos os datasets para treino e verificação.

In [ ]:
raw_df['obito'] = [1 if x == "PACIENTE JÉ ENCONTRADO EM ÓBITO" or x == "ÓBITO DURANTE O ATENDIMENTO" else 0 for x in raw_df['motivo_desfecho']]

madrugada_upper = time(4, 59, 59)
manha_upper = time(11, 59, 59)
tarde_upper = time(17, 59, 59)
noite_upper = time(23, 59, 59)

conditions = [
    (raw_df['hora_minuto'] <= madrugada_upper),
    ((raw_df['hora_minuto'] > madrugada_upper) & (raw_df['hora_minuto'] <= manha_upper)),
    ((raw_df['hora_minuto'] > manha_upper) & (raw_df['hora_minuto'] <= tarde_upper)),
    ((raw_df['hora_minuto'] > tarde_upper) & (raw_df['hora_minuto'] <= noite_upper)),
]
choices = ['madrugada', 'manha', 'tarde', 'noite']
raw_df['periodo'] = np.select(conditions, choices, default='indeterminado')

relevant_columns = [
    "periodo",
    "origem_chamado",
    "tipo",
    "subtipo",
    "sexo",
    "idade",
    "obito"
]
df = raw_df[relevant_columns]
df.head()

In [ ]:
ohe_features = [
    "periodo",
    "origem_chamado",
    "tipo",
    "subtipo",
    "sexo",
]

ohe_df = pd.get_dummies(df, prefix=ohe_features, columns=ohe_features)
ohe_df.head()

In [ ]:
from sklearn.model_selection import train_test_split

features = list(ohe_df.columns)
features.remove('obito')

X = ohe_df[features].to_numpy()

y = ohe_df['obito'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1337)

In [45]:
print("X_train count:", X_train.shape)
print("y_train count:", y_train.shape)
print("X_test count:", X_test.shape)
print("y_test count:", y_test.shape)

X_train count: (69344, 137)
y_train count: (69344,)
X_test count: (23115, 137)
y_test count: (23115,)


# Seleção e otimização de modelos

Os 4 modelos escolhidos foram:
- Naive Bayes (Gaussian)
- Random Forest
- Decision Tree
- Linear SVC

Além disso, utilizamos o método GridSearch para otimização dos hiper-parâmetros.

In [50]:
from sklearn.model_selection import GridSearchCV

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support

labels = [0, 1]

Modelo 1: Naive Bayes

In [52]:
clf = Pipeline([
    ('clf', GaussianNB())
])

parameters = {
    'clf__priors': [None],
    'clf__var_smoothing': np.logspace(0,-9, num=100)
}

gs_clf = GridSearchCV(clf, parameters, cv=5, n_jobs=-1)
gs_clf.fit(X_train, y_train)

for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

print("Stats for optimized GaussianNB")
print("Best Score:", gs_clf.best_score_)
pred = gs_clf.predict(X_test)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, pred, labels=labels)
print("labels:", labels)
print("Precision:", precision)
print("recall:", recall)
print("f1:", f1)

clf__priors: None
clf__var_smoothing: 0.0002310129700083158
Stats for optimized GaussianNB
Best Score: 0.9818585441927112
labels: [0, 1]
Precision: [0.98420159 0.61594203]
recall: [0.9976618  0.18973214]
f1: [0.99088599 0.29010239]


Modelo 2: Random Forest

Modelo 3: Decision Tree

Modelo 4: Linear SVM